## Corona plotting notebook
@jeffluppes (https://github.com/jeffluppes) Mar 17 2020

This is nothing more than a quick and dirty script I spent ~~an hour on~~ that does the following:

* Downloads a public map of Dutch municipalities (Gemeentes)
* Scrapes the RIVM website for data on the daily cases of Covid-19
* Does limited cleaning 
* Stores raw data to your disk as a csv
* Merges the public data with the corona data
* Plots a pretty graph to show where the cases are
* Stores the pretty graph 

Stay safe people, and pay attention to the instructions by the Government and the RIVM. 

In [50]:
import pandas as pd
import geopandas as gpd
import seaborn as sns
import os
import numpy as np
import imageio
import requests
from bs4 import BeautifulSoup
import matplotlib.pyplot as plt
from datetime import date

## The shortest way to obtain a map of the Netherlands

In [21]:
# Retrieve data with municipal boundaries from PDOK
geodata_url = 'https://geodata.nationaalgeoregister.nl/cbsgebiedsindelingen/wfs?request=GetFeature&service=WFS&version=2.0.0&typeName=cbs_gemeente_2017_gegeneraliseerd&outputFormat=json'
municipal_boundaries = gpd.read_file(geodata_url)

## The slightly-longer way to obtain corona data

In [89]:
url="https://www.rivm.nl/coronavirus-kaart-van-nederland"

# Make a GET request to fetch the raw HTML content
html_content = requests.get(url).text

# Parse the html content
soup = BeautifulSoup(html_content, "lxml")

# grab the csv data elment
csvdata = soup.find("div", {"id": "csvData"}).text

# remove the leading \n
csvdata = csvdata[1:]

#store file to disk

# this generates a day stamp - e.g. 17 mar 2020
today = date.today().strftime("%d-%b-%Y")
with open('data/corona_'+today+'.csv','w', encoding='utf-8') as file:
    file.write(csvdata)
    
# this is also the basis for the scraper. I've set this to run on a micro instance on GCP to run every 24hrs at 14:30 

In [93]:
def plot_corona(filename):
    #read corona data from data frame
    corona_df = pd.read_csv(filename, sep=';')
    
    # delete the columns with negative Gemnr
    corona_df = corona_df[(corona_df['Gemnr'] >= 0)]
    
    date = filename.split('_')[1].split('.')[0]

    # delete the columns with negative Gemnr
    corona_df = corona_df[(corona_df['Gemnr'] >= 0)]
    
    # Link data from Statistics Netherlands to geodata
    municipal_corona = pd.merge(municipal_boundaries, corona_df,
                                   left_on = "statnaam", 
                                   right_on = "Gemeente",
                                   how = "outer")
    municipal_corona['Aantal'] = municipal_corona['Aantal'].fillna(0)
    
    # Create a thematic map
    fig, ax = plt.subplots(1, 1, figsize=(16,12))
    p = municipal_corona.plot(column='Aantal',
                                 cmap = 'Oranges',
                                 legend = True,
                                 ax=ax,
                                 figsize = (16,12),
                                 vmax=125,
                                 vmin=1)
    p.axis('off')
    
    p.set_title('Relatieve hoeveelheid Coronagevallen per gemeente op {}'.format(date))
    
    fig.canvas.draw()       # draw the canvas, cache the renderer
    image = np.frombuffer(fig.canvas.tostring_rgb(), dtype='uint8')
    image  = image.reshape(fig.canvas.get_width_height()[::-1] + (3,))

    plt.clf()
    return image

In [94]:
#obtain list of source files
filenames = []

for r, d, f in os.walk('data/'):
    for file in f:
        if ".csv" in file:
            filenames.append(os.path.join(r, file))

In [95]:
kwargs_write = {'fps':1.0, 'quantizer':'nq'}
imageio.mimsave('./corona_spread_netherlands.gif', [plot_corona(f) for f in filenames], fps=1)

<Figure size 1152x864 with 0 Axes>

<Figure size 1152x864 with 0 Axes>

<Figure size 1152x864 with 0 Axes>

<Figure size 1152x864 with 0 Axes>

That's all folks! You now have the below gif made for you.

![](./corona_spread_netherlands.gif)